In [10]:
# random forest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [11]:
# x and y
df = pd.read_csv('preprocessed_data.csv')
label = 'price'
y = df[label]
columns = ['price']
df.drop(columns, inplace=True, axis=1)
feature_namelist = []
for item in df.columns:
    feature_namelist.append(item)
#print(len(feature_namelist))
feature_names = feature_namelist
label = 'price'
x = df[feature_names]

In [12]:
# train and test split
#train_x, test_x, train_y, test_y= train_test_split(x,y,test_size=0.2, random_state=0)

In [13]:
# # metric
# from sklearn.model_selection import cross_val_score
# def rmse_cv(model):
#     rmse = np.sqrt(-cross_val_score(model,x,y,scoring="neg_mean_squared_error", cv=4))
#     return(rmse)

In [14]:
# parameters tuned max_depth and max_features
r2_score_list = []
test_score_list = []
for m in [0,1,2,3]:
    for k in [0.2, 0.3]:
        train_x, test_x, train_y, test_y= train_test_split(x,y,test_size=k, random_state=m)
        for i in [5,25]:
            for j in [3,9]:
                reg = RandomForestRegressor(n_estimators = 100, max_depth = i, min_samples_split = j, random_state = 1000)
                reg.fit(train_x, train_y)
                test_score_result = reg.score(test_x, test_y)
                test_score_list.append(test_score_result)
                #print('train_score:',reg.score(train_x, train_y))
                #print('test_score:',reg.score(test_x, test_y))
                y_pred = reg.predict(test_x)#.reshape((9779,1))
                r2_score_result = r2_score(test_y, y_pred)
                r2_score_list.append(r2_score_result)

print('r2_score:', max(r2_score_list))
print('std of test_score:', np.std(test_score_list))
print('mean of test_score:', np.mean(test_score_list))

r2_score: 0.19137090819436897
std of test_score: 0.03204378836537754
mean of test_score: 0.12203242204528361


In [15]:
#print('r2_score for random_forest:', max(r2_score_list))

In [ ]:
nr_runs = 10
ftr_names = feature_names
scores = np.zeros([len(ftr_names),nr_runs])

test_score = reg.score(test_x, test_y)
#print('test score = ',test_score)
#print('test baseline = ',np.sum(test_y == 0)/len(test_y))
# loop through the features
for i in range(len(ftr_names)):
    print('shuffling '+str(ftr_names[i]))
    acc_scores = []
    for j in range(nr_runs):
        X_test_shuffled = test_x.copy()
        X_test_shuffled[ftr_names[i]] = np.random.permutation(test_x[ftr_names[i]].values)
        acc_scores.append(reg.score(X_test_shuffled,test_y))
    print('   shuffled test score:',np.around(np.mean(acc_scores),3),'+/-',np.around(np.std(acc_scores),3))
    scores[i] = acc_scores

shuffling neighbourhood_group_Bronx
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_group_Brooklyn
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_group_Manhattan
   shuffled test score: 0.11 +/- 0.001
shuffling neighbourhood_group_Queens
   shuffled test score: 0.105 +/- 0.001
shuffling neighbourhood_group_Staten Island
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Allerton
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Arden Heights
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Arrochar
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Arverne
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Astoria
   shuffled test score: 0.103 +/- 0.003
shuffling neighbourhood_Bath Beach
   shuffled test score: 0.105 +/- 0.0
shuffling neighbourhood_Battery Park City
   shuffled test score: 0.11 +/- 0.003
shuffling neighbourhood_Bay Ridge
   shuffled test score: 0.103 +/- 0.0
shuffling neighbourhood_B

In [ ]:
sorted_indcs = np.argsort(np.mean(scores,axis=1))[::-1]
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(8,6))
plt.boxplot(scores[sorted_indcs].T,labels=ftr_names[sorted_indcs],vert=False)
plt.axvline(test_score,label='test score')
plt.title("Permutation Importances (test set)")
plt.xlabel('score with perturbed feature')
plt.legend()
plt.tight_layout()
plt.show()